<a href="https://colab.research.google.com/github/priyanshu8493/Skin_Disease-Predictor-App/blob/main/Skin_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# downloading data via kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("riyaelizashaju/skin-disease-classification-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 169M/169M [00:01<00:00, 109MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/riyaelizashaju/skin-disease-classification-image-dataset/versions/1


In [3]:
# getting the directories to train and val data
import os

dataset_dir = '/root/.cache/kagglehub/datasets/riyaelizashaju/skin-disease-classification-image-dataset/versions/1'

for root, dirs, _ in os.walk(dataset_dir):
  if 'train' in dirs:
    train_dir = os.path.join(root, 'train')
  if 'val' in dirs:
    val_dir = os.path.join(root,'val')


print(train_dir)
print(val_dir)


# image size = (600,450)


/root/.cache/kagglehub/datasets/riyaelizashaju/skin-disease-classification-image-dataset/versions/1/Split_smol/train
/root/.cache/kagglehub/datasets/riyaelizashaju/skin-disease-classification-image-dataset/versions/1/Split_smol/val


In [23]:
# setting up the datagenerator from train and val directories

from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_ins = ImageDataGenerator( rescale = 1.0/255,
                             width_shift_range = 0.2,
                              height_shift_range = 0.2,         # added image augmentations to add more images to our set
                              rotation_range = 40,
                              shear_range = 0.2,
                              horizontal_flip = True,
                              zoom_range = 0.2,
                              fill_mode = 'nearest')

train_generator = gen_ins.flow_from_directory(train_dir, class_mode = 'categorical', target_size = (600,600))

val_generator = gen_ins.flow_from_directory(val_dir, class_mode = 'categorical', target_size = (600,600))


# all images rescaled to (400,400) size which is the input shape to our cnn

Found 697 images belonging to 9 classes.
Found 181 images belonging to 9 classes.


In [9]:
#EXPERIMENT
#checking the input shape of an image

from PIL import Image
import os
image_path = 'ISIC_0024707.jpg'

for root, dirs, _ in os.walk(dataset_dir):
  if image_path in _:
    image_path = os.path.join(root, image_path)

with Image.open(image_path) as img:
  print(img.size)




(600, 450)


In [24]:
# defining the model
import tensorflow as tf

#creating a cnn

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (600,600,3) ),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),   # added some dropout regulation layers to prevent overfitting
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation ='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense( 9, activation = 'softmax')
])

model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#fitting the model

history = model.fit(train_generator, epochs = 50, validation_data = val_generator)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 509s 22s/step - accuracy: 0.1457 - loss: 22.8108 - val_accuracy: 0.1602 - val_loss: 2.1530
Epoch 2/50
17/22 ━━━━━━━━━━━━━━━━━━━━ 1:30 18s/step - accuracy: 0.1647 - loss: 2.1129